In [34]:
import torch
import torch.nn as nn
from ctc_loss_imp import ctc_loss_imp

In [74]:
device    = 'cpu'
T  = 40
C_SIZE = 5

tgt_len = [5,5,5,5]
inp_len = [T,T,T,T]
BS = len(tgt_len)

targets = torch.randint(BS, C_SIZE , (sum(tgt_len),), dtype=torch.int)
log_probs = torch.nn.Parameter((torch.randn(T, BS, C_SIZE, dtype=torch.float, device=device).softmax(2))).log()

ctc_loss = torch.nn.CTCLoss()
ctc_loss_custom = ctc_loss_imp_log_space_2
out_built_in = ctc_loss(log_probs, targets, inp_len, tgt_len) # in log space
out_custom = ctc_loss_custom(log_probs, targets, inp_len, tgt_len)  # in soft max space

print(out_built_in, out_custom)
assert torch.norm(out_built_in - out_custom) < 1e-4

tensor(10.0246, grad_fn=<MeanBackward0>)
tensor(10.0246, grad_fn=<MeanBackward0>) tensor(10.0246, grad_fn=<MeanBackward0>)


In [73]:

neginf    = torch.tensor(-float('inf'))
def ctc_loss_imp_log_space_2(inp, targets, inp_len, tgt_len, BLANK=0, reduction='mean'):
        # inp in log space
        inp_len       = torch.as_tensor(inp_len, dtype=torch.int)
        tgt_len       = torch.as_tensor(tgt_len, dtype=torch.int)
        cum_tgt_len   = tgt_len.cumsum(0)
        losses        = []
        alpha_global  = []

        for i in range(inp.size(1)):
            inp_length          = inp_len[i].item()
            target_length       = tgt_len[i].item()
            cum_target_length   = cum_tgt_len[i].item()

            targets_prime = targets.new_full((2 * target_length + 1,), BLANK)
            if targets.dim() == 2:
                targets_prime[1::2] = targets[i, :target_length]
            else:
                targets_prime[1::2] = targets[cum_target_length - target_length:cum_target_length]

            probs = inp[:inp_length, i]
            alpha   = inp.new_ones((inp_length, target_length*2+1)) * neginf 
            alpha[0,0] = probs[0, BLANK]
            if target_length > 0:
                alpha[0,1] = probs[0, targets_prime[1]]

            for t in range(1, inp_length):
                for s in range(2 * target_length +1):
                    a1 = alpha[t-1,s]
                    a2 = alpha[t-1,s-1] if s > 0 else neginf
                    a3 = alpha[t-1,s-2] if s > 1 and targets_prime[s-2]!= targets_prime[s] else neginf

                    amax = max(a1,a2,a3)
                    amax = 0 if amax == neginf else amax
                    
                    alpha[t,s] = torch.log( torch.exp(a1-amax) + torch.exp(a2-amax) + torch.exp(a3-amax)) + \
                        amax + probs[t, targets_prime[s]]

            if target_length == 0:
                loss = -alpha[-1,0]
            else:
                l1 = alpha[-1,-2]
                l2 = alpha[-1,-1]
                loss = -torch.log(torch.exp(l1)+ torch.exp(l2))
            losses.append(loss[None])
            alpha_global.append(alpha)
        output = torch.cat(losses, 0)
        if reduction == 'mean':
            print( (output / tgt_len.to(dtype=output.dtype, device=output.device)).mean() )
        elif reduction == 'sum':
            print( output.sum() )

        output_mean = (output / tgt_len.to(dtype=output.dtype, device=output.device)).mean()

        return output_mean

In [66]:

loss_imp = ctc_loss_imp_log_space_2(input, target, input_lengths, target_lengths)

tensor(inf, grad_fn=<MeanBackward0>)
